In [1]:
import numpy as np
from matplotlib import pyplot as plt
import random as rand
from math import floor
from IPython.display import clear_output, display

# Helper function definitions
def check_neighbors(i, j, map_to_check, include_center):
    num_filled_neighbors = 0
    for x in range(max(0, i-1), min(map_to_check.shape[0], i + 2)):
        for y in range(max(0, j-1), min(map_to_check.shape[1], j + 2)):
            if map_to_check[x][y] != 0:
                num_filled_neighbors += 1
    if not include_center and map_to_check[i][j] != 0:
        num_filled_neighbors -= 1
    return num_filled_neighbors

def find_max_neighbor(i, j, map_to_check): 
    max_neighbor = [[i, j]]
    for x in range(max(0, i-1), min(map_to_check.shape[0], i + 2)):
        for y in range(max(0, j-1), min(map_to_check.shape[1], j + 2)):
            if map_to_check[x][y] > map_to_check[max_neighbor[0][0]][max_neighbor[0][1]]:
                max_neighbor = [[x, y]]
            elif map_to_check[x][y] == map_to_check[max_neighbor[0][0]][max_neighbor[0][1]]:
                max_neighbor.append([x, y])
    return rand.choice(max_neighbor)

def update_plants(plant_map):
    mask = np.zeros(plant_map.shape)
    for i in range(plant_map.shape[0]):
        for j in range(plant_map.shape[1]):
            if check_neighbors(i, j, plant_map, True) > 0:
                mask[i][j] = 1
    plant_map += mask
    return plant_map

def update_herbs(herb_map, plant_map, num_moves, hunger, reproduction_chance, density_limit):
    excess_food_map = plant_map - herb_map

    for _ in range(num_moves):
        movement_mask = np.zeros(plant_map.shape)
        for i in range(herb_map.shape[0]):
            for j in range(herb_map.shape[1]):
                if herb_map[i][j] > plant_map[i][j]:
                    amount = herb_map[i][j] - plant_map[i][j]
                    dest = find_max_neighbor(i, j, excess_food_map)
                    movement_mask[i][j] -= amount
                    movement_mask[dest[0]][dest[1]] += amount
        herb_map += movement_mask

    reproduction_mask = np.zeros(herb_map.shape)
    for i in range(herb_map.shape[0]):
        for j in range(herb_map.shape[1]):
            for _ in range(int(herb_map[i][j])):
                if rand.random() < reproduction_chance:
                    reproduction_mask[i][j] += 1

    for i in range(herb_map.shape[0]):
        for j in range(herb_map.shape[1]):
            for _ in range(int(herb_map[i][j])):
                if rand.randint(0, 10) < hunger:
                    plant_map[i][j] = max(0, plant_map[i][j] - 1)

    herb_map += reproduction_mask
    herb_map = np.clip(herb_map, 0, density_limit)

    return herb_map, plant_map

# Initialize the map
island_size = [20, 20]
plants_start = 5
plants = np.ones(island_size) * plants_start
herbs = np.zeros(island_size)
carns = np.zeros(island_size)
plants_line = []
herbs_line = []
carns_line = []

herbs[0:4, :] = 5
carns[0, :] = 2
fig, ax_list = plt.subplots(1, 4, figsize=(15, 3.5))

sim_length = 100
for i in range(sim_length):  # Main update loop
    ax_list[0].cla()
    ax_list[1].cla()
    ax_list[2].cla()
    ax_list[3].cla()
    old_herbs = np.array([herbs])
    old_carns = np.array([carns])
    plants = update_plants(plants)
    herbs, plants = update_herbs(herbs, plants, 5, 7, 0.2, 8)  # Adjust reproduction_chance as needed
    carns, herbs = update_herbs(carns, herbs, 5, 2, 0.1, 10)  # Adjust reproduction_chance as needed
    clear_output(wait=True)
    ax_list[0].imshow(plants)
    ax_list[0].set_title("Plants: " + str(int(np.sum(plants))))
    ax_list[1].imshow(herbs)
    ax_list[1].set_title("Herbivores: " + str(int(np.sum(herbs))))
    ax_list[2].imshow(carns)
    ax_list[2].set_title("Carnivores: " + str(int(np.sum(carns))))
    ax_list[3].set_xlim(0, sim_length)
    ax_list[3].set_ylim(0, island_size[0] * island_size[1] * plants_start)
    ax_list[3].plot(plants_line, 'g')
    ax_list[3].plot(herbs_line, 'b')
    ax_list[3].plot(carns_line, 'r')
    plt.show()
    display(fig)

    plants_line.append(np.sum(plants))
    herbs_line.append(np.sum(herbs))
    carns_line.append(np.sum(carns))

    if np.count_nonzero(old_herbs - np.array([herbs])) == 0 and np.count_nonzero(old_carns - np.array([carns])) == 0:
        break

print("DONE")


KeyboardInterrupt: 